In [6]:
import os, importlib, numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display
import time as time 

import classA_U1FGTN as mod
importlib.reload(mod)
from classA_U1FGTN import classA_U1FGTN

In [7]:
# ---------------- config ----------------
Nx, Ny        = 16, 16
Ntot          = 4*Nx*Ny
Nlayer        = Ntot//2
cycles        = 20
samples       = 1
nshell        = None
DW            = True
filling_frac  = 0.5
backend       = "threading"   # safer in constrained environments
progress      = True
n_a           = 0.5
init_mode     = 'maxmix'
model         = classA_U1FGTN(Nx=Nx, Ny=Ny, DW=DW, nshell=nshell, filling_frac=filling_frac, G0=None)
model.construct_OW_projectors(nshell=nshell, DW=DW)
if init_mode == "default":
    G_init = model.random_complex_fermion_covariance(N = Nlayer)
elif init_mode == "maxmix":
    G_init = np.zeros((Nlayer, Nlayer))

DWs at x=(4, 11)
------------------------- classA_U1FGTN Initialized -------------------------


In [8]:
Ilayer = np.eye(Nlayer)

def G2pt_from_G(G):
    return 0.5*(G + Ilayer)

def ancilla_swap(Gtop, P, n_a=0.5):
    Q = Ilayer - P
    Gnew = Q @ Gtop @ Q + (2*n_a-1) * P
    return 0.5 * (Gnew + Gnew.conj().T)

In [9]:
P_Ap = model._proj_from_WF(model.WF_Ap, 0, 0)
P_Bp = model._proj_from_WF(model.WF_Bp, 0, 0)
P_Am = model._proj_from_WF(model.WF_Am, 0, 0)
P_Bm = model._proj_from_WF(model.WF_Bm, 0, 0)
P_dict = {'Ap' : P_Ap, 'Bp' : P_Bp, 'Am' : P_Am, 'Bm' : P_Bm}

In [15]:
chi_Ap = model.WF_Ap[:,0,0]
chi_Bp = model.WF_Bp[:,0,0]
chi_Am = model.WF_Am[:,0,0]
chi_Bm = model.WF_Bm[:,0,0]

In [25]:
# testing markov feedback
for lbl, P in P_dict.items():
    p_occ_before = np.real(np.trace(G2pt_from_G(G_init) @ P))
    print(f'{lbl}, p_occ_before : {p_occ_before}')
    p_occ_after = np.real(np.trace(G2pt_from_G(ancilla_swap(G_init, P)) @ P))
    print(f'{lbl}, p_occ_after : {p_occ_after}')


Ap, p_occ_before : 0.5131046996827036
Ap, p_occ_after : 0.5000000000000003
Bp, p_occ_before : 0.5416700226220835
Bp, p_occ_after : 0.5000000000000003
Am, p_occ_before : 0.5122704097675825
Am, p_occ_after : 0.5000000000000007
Bm, p_occ_before : 0.5263842547598471
Bm, p_occ_after : 0.5000000000000008


In [13]:
Il = np.eye(Nlayer, dtype=np.complex128)

def _measure(Gtop, P, particle):
    if particle:
        H11, H21, H22 = -P, (Il - P), P
    else:
        H11, H21, H22 = P, (Il - P), -P
    inv_time_start = time.time()
    Ginv = np.linalg.pinv(Gtop, rcond=1e-9, hermitian=True)
    print(f'inv_time : {time.time()-inv_time_start}')
    K = H11 - Ginv
    H12 = H21.conj().T
    solve_time = time.time()
    X = model._solve_regularized(K, H12, eps=1e-9)
    print(f'solve_regularized_time : {time.time()-solve_time}')
    G_upd = H22 - H21 @ X
    return 0.5 * (G_upd + G_upd.conj().T)

In [17]:
def _measure_v2(G, P, particle):
    Q = Il - P

    start = time.time()
    if particle:
        G_upd = P + Q @ (G - G @ P @ G / (1+np.trace(G @ P)))
    else:
        G_upd = -P + Q @ (G + G @ P @ G / (1-np.trace(G @ P)))
    print(f'time = {time.time()-start}')

    return 0.5 * (G_upd + G_upd.conj().T)

In [19]:
G_init_prime = np.zeros((Nlayer, Nlayer))
G = _measure_v2(G_init_prime, P_Ap, particle=False)
p_occ = np.real(np.trace(G2pt_from_G(G) @ P_Ap))
print(p_occ)
Gprime = ancilla_swap(G, P_Ap, n_a=1)
p_occ_prime = np.real(np.trace(G2pt_from_G(Gprime) @ P_Ap))

print(f'p_occ : {p_occ}')
print(f'p_occ_prime : {p_occ_prime}')

time = 0.0182037353515625
-3.26708142794246e-16
p_occ : -3.26708142794246e-16
p_occ_prime : 1.0000000000000007
